In [2]:
%matplotlib 
%load_ext autoreload
%autoreload 2

Using matplotlib backend: Qt4Agg


In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
import sys
sys.path.append('/home/chenani/ownCloud/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/')

In [4]:
import matplotlib
import signale
import numpy as np
import matplotlib.pyplot as pl
import spectrum as sp
import pickle as pkl
import scipy.signal as scsig
import pywt
import colormaps as mycmps
import signale.tools as tools
import os
import itertools
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
from sklearn.cluster import KMeans
from matplotlib.mlab import PCA as mlabPCA

/home/chenani/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
/home/chenani/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [5]:
def MA(array,ord=2):
    ma = np.array([array[ii-ord/2:ii+ord/2].mean() for ii in range(array.size) if ii >= ord ],dtype=float)
    ma = np.insert(ma,0,array[0:ord/2])
    ma = np.append(ma,array[-(ord/2)-1:-1])
    return ma
def zScore(array):
    return(array - np.average(array))/ np.std(array)
def mySpecgram(sig,minFreq = None,maxFreq = None,overlap=None,
               windowSize= None,NFFT = None,Fs = None,method=None):
    if not overlap:
        overlap = 48
    kIdx = 0
    timeSlices = []
    timeCenters = []
    while kIdx < sig.size:
        #timeEdges = np.arange(0,Nbins*windowSize,windowSize)
        timeSlices.append([kIdx,kIdx+windowSize])
        kIdx += windowSize - overlap
    Pxx = []
    #print timeSlices
    for item in timeSlices:
        timeCenters.append((item[0]+np.diff(item)/2)[0])
        data = sig[item[0]:item[1]]
        if data.size < windowSize:
            ddd = windowSize - data.size
            if ddd%2 ==0:
                data = np.lib.pad(data, (ddd/2,ddd/2), 'constant', constant_values=(0,0))
            else:
                data = np.lib.pad(data, (ddd/2,ddd/2+1), 'constant', constant_values=(0,0))
        if method == 'yule':
            p = sp.pyule(data, 550, norm='biased', NFFT=NFFT,sampling=Fs)
        elif method =='fft':
            p = sp.Periodogram(data,NFFT=NFFT,sampling=Fs)
        p();
        freqz = np.array(p.frequencies())
        Pxx.append(p.psd)
    Pxx = np.array(Pxx).T
    if not minFreq:
        minFreq = freqz.min()
    if not maxFreq:
        maxFreq = freqz.max()
    indexstart = np.where(freqz >= minFreq)[0][0]
    indexend   = np.where(freqz <= maxFreq)[0][-1]
    freqz = freqz[indexstart:indexend]
    Pxx = Pxx[indexstart:indexend]
    timeCenters = np.array(timeCenters)
    return timeCenters,freqz,Pxx


In [6]:
animalPath = '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3839/LinearTrack_12-23-15_Day3/'
ddayx = animalPath.split('/')[-2].split('_')[-1]
#print dday
dataFolder = '/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3839Control/dataFiles/sleepDB/'

In [7]:
lfpPaths = []
for lfp in tools.locate('*-p*.lfp',animalPath):#animalPath):
    lfpPaths.append(os.path.join(lfp[0],lfp[1]))
lfpPaths = sorted(lfpPaths)
ImmoDBpaths = []
for db in tools.locate('*.db',dataFolder):
    if db[1].find(ddayx)>0:
        ImmoDBpaths.append(os.path.join(db[0],db[1]))
lfpPaths = sorted(lfpPaths)
ImmoDBpaths = sorted(ImmoDBpaths)
lfpPaths,ImmoDBpaths

(['/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3839/LinearTrack_12-23-15_Day3/01-sleep1/CSC5-pre.lfp',
  '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3839/LinearTrack_12-23-15_Day3/03-sleep2/CSC5-post.lfp'],
 ['/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3839Control/dataFiles/sleepDB/ImmPeriods-Day3-01pre.db',
  '/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3839Control/dataFiles/sleepDB/ImmPeriods-Day3-03post.db'])

In [8]:
SleepEpochsDB = pd.DataFrame([],columns=['t0','t1','epoch','session','date','animal'])
for item,jtem in zip(ImmoDBpaths,lfpPaths):
    print 'LOADING %s' %jtem
    ##Loading...
    expDate = jtem.split('_')[-2]
    session = jtem.split('/')[-2].split('-')[-1]
    session = session[:-1]+'0'+session[-1]
    animal = [prt for prt in jtem.split('/') if not prt.find('Rat')][0]
    SleepTimeIndex = pd.read_pickle(item)
    lfp = pkl.load(open(jtem,'rb'))
    t,frq,Pxx =  mySpecgram(lfp.signal,minFreq=0.1,maxFreq=20,windowSize=2048,method='fft',Fs=2000,overlap=1024,NFFT=4096)
    Pxx = np.clip(Pxx,0,Pxx.mean()+1.4*Pxx.std())
    pxxFlat = np.array([Pxx[:,ii]*frq for ii in range(Pxx.shape[1])]).T
    for jjj in range(pxxFlat.shape[0]):
        pxxFlat[jjj,:] = MA(pxxFlat[jjj,:],14)
    
    ##Grouping into different freqency bands, delta,theta,gamma
    Dmean = frq[frq<3.5].mean()
    Tmean = frq[np.logical_and(frq>6,frq<10)].mean()
    Gmean = frq[frq>12].mean()
    Dbounds = np.where(frq<3.5)[0][[0,-1]]
    Tbounds = np.where(np.logical_and(frq>6,frq<10))[0][[0,-1]]
    Gbounds = np.where(frq>12)[0][[0,-1]]
    DavgPwr = Pxx[Dbounds[0]:Dbounds[-1]+1,:].mean(axis=0)
    TavgPwr = Pxx[Tbounds[0]:Tbounds[-1]+1,:].mean(axis=0)
    GavgPwr = Pxx[Gbounds[0]:Gbounds[-1]+1,:].mean(axis=0)
    #DpwrNormal = DavgPwr*Dmean
    #TpwrNormal = TavgPwr*Tmean
    #GpwrNormal = GavgPwr*Gmean
    DpwrNormal = pxxFlat[Dbounds[0]:Dbounds[-1]+1,:].mean(axis=0)
    TpwrNormal = pxxFlat[Tbounds[0]:Tbounds[-1]+1,:].mean(axis=0)
    GpwrNormal = pxxFlat[Gbounds[0]:Gbounds[-1]+1,:].mean(axis=0)
    ###Clustering states using k-means
    pwrData = np.array([DpwrNormal,TpwrNormal,GpwrNormal]).T
    pwrPCA= mlabPCA(pwrData)
    Klusters = KMeans(n_clusters=2).fit_predict(pwrData)
    Klusters = KMeans(n_clusters=2).fit_predict(pwrPCA.Y)
    k2 = Klusters.copy()
    print 1.0*np.where(Klusters==1)[0].size/Klusters.size
    #######################################################################################################################pl.plot(k2)
    UpIdx   = np.where(Klusters==1)[0]
    DownIdx = np.where(Klusters==0)[0]
    SingletIdxU = np.where(np.diff(DownIdx)==2)[0]
    SingletIdxD = np.where(np.diff(UpIdx)  ==2)[0]
    while SingletIdxU.size or SingletIdxD.size:
        print SingletIdxU.size,SingletIdxD.size
        if UpIdx.size > DownIdx.size:
            SingletIdxU = np.int32(DownIdx[SingletIdxU]  + np.ones(SingletIdxU.size))
            np.put(Klusters,SingletIdxU,np.zeros(SingletIdxU.size))
            SecondFlip =False
        else:
            SingletIdxD = np.where(np.diff(UpIdx)==2)[0]
            SingletIdxD  = np.int32(UpIdx[SingletIdxD]  + np.ones(SingletIdxD.size))
            np.put(Klusters,SingletIdxD,np.ones(SingletIdxD.size))
            SecondFlip =True
        if SecondFlip:
            SingletIdxU = np.where(np.diff(DownIdx)==2)[0]
            SingletIdxU  = np.int32(DownIdx[SingletIdxU]  + np.ones(SingletIdxU.size))
            np.put(Klusters,SingletIdxU,np.zeros(SingletIdxU.size))
        else:
            SingletIdxD = np.where(np.diff(UpIdx)==2)[0]
            SingletIdxD  = np.int32(UpIdx[SingletIdxD]  + np.ones(SingletIdxD.size))
            np.put(Klusters,SingletIdxD,np.ones(SingletIdxD.size))
        del SecondFlip
        ###########222#######
        ###
        ###########222#######
        K1ratio = (DpwrNormal[Klusters==1] / TpwrNormal[Klusters==1]).mean()
        K0ratio = (DpwrNormal[Klusters==0] / TpwrNormal[Klusters==0]).mean()
        if K1ratio<K0ratio:
            print 'Cluster labels fliped!!!'
            print K1ratio,K0ratio
            Klusters = np.int8(np.logical_not(Klusters))
        k1 = Klusters.copy()
        ###
        # from now on SWS episodes are Kluster ==1
        ###
        swsIdx = np.where(Klusters==1)[0]
        dupletIdx = np.where(np.diff(swsIdx)==3)[0]
        dupletIdx1st = np.int32(swsIdx[dupletIdx] + np.ones(dupletIdx.size))
        dupletIdx2nd = np.int32(swsIdx[dupletIdx] +2* np.ones(dupletIdx.size))
        np.put(Klusters,dupletIdx1st,np.ones(dupletIdx.size))
        np.put(Klusters,dupletIdx2nd,np.ones(dupletIdx2nd.size))
        ####Repeat the same procedure for REM
        remIdx = np.where(Klusters==0)[0]
        dupletIdx = np.where(np.diff(remIdx)==3)[0]
        dupletIdx1st = np.int32(remIdx[dupletIdx] + np.ones(dupletIdx.size))
        dupletIdx2nd = np.int32(remIdx[dupletIdx] +2* np.ones(dupletIdx.size))
        np.put(Klusters,dupletIdx1st,np.zeros(dupletIdx.size))
        np.put(Klusters,dupletIdx2nd,np.zeros(dupletIdx2nd.size))
        ##########333############
        ###Now chunks of length 3!
        ##########333############
        swsIdx = np.where(Klusters==1)[0]
        dupletIdx = np.where(np.diff(swsIdx)==4)[0]
        dupletIdx1st = np.int32(swsIdx[dupletIdx] + np.ones(dupletIdx.size))
        dupletIdx2nd = np.int32(swsIdx[dupletIdx] +2* np.ones(dupletIdx.size))
        dupletIdx3rd = np.int32(swsIdx[dupletIdx] +3* np.ones(dupletIdx.size))
        np.put(Klusters,dupletIdx1st,np.ones(dupletIdx.size))
        np.put(Klusters,dupletIdx2nd,np.ones(dupletIdx2nd.size))
        np.put(Klusters,dupletIdx3rd,np.ones(dupletIdx3rd.size))
        ####Repeat the same procedure for REM
        remIdx = np.where(Klusters==0)[0]
        dupletIdx = np.where(np.diff(remIdx)==4)[0]
        dupletIdx1st = np.int32(remIdx[dupletIdx] + np.ones(dupletIdx.size))
        try:
            dupletIdx2nd = np.int32(remIdx[dupletIdx] +2* np.ones(dupletIdx.size))
            dupletIdx3rd = np.int32(swsIdx[dupletIdx] +3* np.ones(dupletIdx.size))
        except IndexError:
            print 'It\'s a bit marginal!'
        np.put(Klusters,dupletIdx1st,np.ones(dupletIdx.size))
        np.put(Klusters,dupletIdx2nd,np.ones(dupletIdx2nd.size))
        np.put(Klusters,dupletIdx3rd,np.ones(dupletIdx3rd.size))
        ##########444############
        ###Now chunks of length 4!
        ##########444############
        swsIdx = np.where(Klusters==1)[0]
        dupletIdx = np.where(np.diff(swsIdx)==5)[0]
        dupletIdx1st = np.int32(swsIdx[dupletIdx] + np.ones(dupletIdx.size))
        dupletIdx2nd = np.int32(swsIdx[dupletIdx] +2* np.ones(dupletIdx.size))
        dupletIdx3rd = np.int32(swsIdx[dupletIdx] +3* np.ones(dupletIdx.size))
        dupletIdx4th = np.int32(swsIdx[dupletIdx] +4* np.ones(dupletIdx.size))
        np.put(Klusters,dupletIdx1st,np.ones(dupletIdx.size))
        np.put(Klusters,dupletIdx2nd,np.ones(dupletIdx2nd.size))
        np.put(Klusters,dupletIdx3rd,np.ones(dupletIdx3rd.size))
        np.put(Klusters,dupletIdx4th,np.ones(dupletIdx4th.size))
        ####Repeat the same procedure for REM
        remIdx = np.where(Klusters==0)[0]
        dupletIdx = np.where(np.diff(remIdx)==5)[0]
        dupletIdx1st = np.int32(remIdx[dupletIdx] + np.ones(dupletIdx.size))
        try:
            dupletIdx2nd = np.int32(remIdx[dupletIdx] +2* np.ones(dupletIdx.size))
            dupletIdx3rd = np.int32(swsIdx[dupletIdx] +3* np.ones(dupletIdx.size))
            dupletIdx4th = np.int32(swsIdx[dupletIdx] +4* np.ones(dupletIdx.size))
        except IndexError:
            print 'It\'s a bit marginal!'
        np.put(Klusters,dupletIdx1st,np.ones(dupletIdx.size))
        np.put(Klusters,dupletIdx2nd,np.ones(dupletIdx2nd.size))
        np.put(Klusters,dupletIdx3rd,np.ones(dupletIdx3rd.size))
        np.put(Klusters,dupletIdx4th,np.ones(dupletIdx4th.size))
                
        UpIdx   = np.where(Klusters==1)[0]
        DownIdx = np.where(Klusters==0)[0]
        SingletIdxU = np.where(np.diff(DownIdx)==2)[0]
        SingletIdxD = np.where(np.diff(UpIdx)  ==2)[0]
    print 1.0*np.where(Klusters==1)[0].size/Klusters.size
    ####################################################
    SleepEpochs = pd.DataFrame([],columns=['t0','t1','epoch','session','date','animal'])
    fallDowns = np.where(np.diff(Klusters)==-1)[0]
    jumpUps   = np.where(np.diff(Klusters)== 1)[0]
    if fallDowns[0]<jumpUps[0]:
        for ii in range(fallDowns.size):
            try:
                df = pd.DataFrame([[t[fallDowns[ii]]/2,t[jumpUps[ii]-1]/2,'REM',session,expDate,animal],
                                   [t[jumpUps[ii]]/2,t[fallDowns[ii+1]-1]/2,'SWS',session,expDate,animal]],
                                  columns=['t0','t1','epoch','session','date','animal'])
            except IndexError:
                print 'Oh'
            SleepEpochs = SleepEpochs.append(df,ignore_index=True)
    else:
        for ii in range(jumpUps.size):
            try:
                df = pd.DataFrame([[t[jumpUps[ii]]/2,t[fallDowns[ii]-1]/2,'SWS',session,expDate,animal],
                                   [t[fallDowns[ii]]/2,t[jumpUps[ii+1]-1]/2,'REM',session,expDate,animal]],
                                  columns=['t0','t1','epoch','session','date','animal'])
            except IndexError:
                print 'Oh'
            SleepEpochs = SleepEpochs.append(df,ignore_index=True)
    SleepEpochsDB = SleepEpochsDB.append(SleepEpochs,ignore_index=True)
    #############################################################################################
    f,ax=pl.subplots(1,1)
    ax.pcolormesh(t/2,frq,np.log10(pxxFlat),cmap='magma')
    ax.plot(lfp.timeAxis-lfp.timeAxis[0],zScore(lfp.signal)+7,lw=1)
    for idx,row in SleepEpochs.iterrows():
        if row.epoch == 'SWS':
            ax.axvspan(row.t0,row.t1,ymin=0.9,ymax=0.95,color='r')
        else:
            ax.axvspan(row.t0,row.t1,ymin=0.8,ymax=0.85)
    ax.plot(t/2,Klusters,lw=3)
    #############################################################################################

LOADING /home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3839/LinearTrack_12-23-15_Day3/01-sleep1/CSC5-pre.lfp
0.826381365114
0 10
0.829767063922
Oh
LOADING /home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3839/LinearTrack_12-23-15_Day3/03-sleep2/CSC5-post.lfp
0.656241273387
28 36
Cluster labels fliped!!!
0.685385449838 0.854432539112
It's a bit marginal!
0 1
It's a bit marginal!
0.348645629712
Oh


/home/chenani/anaconda/lib/python2.7/site-packages/spectrum-0.6.1-py2.7-linux-x86_64.egg/spectrum/psd.py:451: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if self.__psd == None:


In [16]:
day = [item.split('/')[0] for item in animalPath.split('_') if not item.find('Day')][0]
SleepEpochsDB.to_pickle(dataFolder+'SleepEpoch-'+animal+'-'+day+'.db')

In [9]:
SleepEpochsDB

,t0,t1,epoch,session,date,animal
0,388608,394752,REM,sleep01,12-23-15,Rat3839
1,395264,401920,SWS,sleep01,12-23-15,Rat3839
2,402432,427520,REM,sleep01,12-23-15,Rat3839
3,428032,497664,SWS,sleep01,12-23-15,Rat3839
4,498176,542720,REM,sleep01,12-23-15,Rat3839
5,543232,551424,SWS,sleep01,12-23-15,Rat3839
6,551936,586240,REM,sleep01,12-23-15,Rat3839
7,586752,597504,SWS,sleep01,12-23-15,Rat3839
8,598016,611328,REM,sleep01,12-23-15,Rat3839
9,611840,626688,SWS,sleep01,12-23-15,Rat3839


In [17]:
for idx,item in SleepEpochsDB.iterrows():
    if (item.epoch == 'SWS' and item.session=='sleep02'):
        pl.axvspan(item.t0,item.t1,alpha==0.1)
pl.plot(lfp.timeAxis-lfp.timeAxis[0],lfp.signal)

REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
REM
SWS
SWS


NameError: name 'alpha' is not defined